In [1]:
import torch
import numpy
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt

## 데이터 생성 및 시각화

In [2]:
# 2차원 데이터
n_dim = 2
# 샘플 데이터를 만들어주는 함수
x_train, y_train = make_blobs(
    n_samples=80, n_features=n_dim,
    centers=[[0,0],[1,1],[1,0],[0,1]],
    shuffle=True, cluster_std=0.15
)
x_test, y_test = make_blobs(
    n_samples=20, n_features=n_dim,
    centers=[[0,0],[1,1],[1,0],[0,1]],
    shuffle=True, cluster_std=0.15
)

In [3]:
# 데이터 초반부 훑어보기
for x, y in list(zip(x_train, y_train))[:10]:
    print(f'x: {str(x):30} y: {y}')

x: [0.85733237 1.02898188]        y: 1
x: [1.1763867  0.83389475]        y: 1
x: [0.07649034 0.03095951]        y: 0
x: [1.01768332 0.93344804]        y: 1
x: [0.20791738 0.21656507]        y: 0
x: [0.05692194 1.1657096 ]        y: 3
x: [0.28836858 0.05401223]        y: 0
x: [1.08809739 1.06455869]        y: 1
x: [0.90654332 1.08861466]        y: 1
x: [1.00059178 0.88745697]        y: 1


In [4]:
# 이진 분류를 할 것이므로 생성한 데이터의 0,1번 클래스는 0으로, 2,3번 클래스는 1로 재설정
label_dict = {
    0:0, 1:0,
    2:1, 3:1
}
y_train = np.vectorize(lambda l: label_dict[l])(y_train)
y_test = np.vectorize(lambda l: label_dict[l])(y_test)

NameError: ignored

In [ ]:
def draw(x, y, style='o'):
    for x_, y_ in zip(x,y):
        plt.plot(x_[0], x_[1], 'b'+style if y_ == 0 else 'r'+style)

plt.figure()
# 데이터 생성 기준선
plt.hlines(0, -2, 2, color='k', linestyles=':')
plt.hlines(1, -2, 2, color='k', linestyles=':')
plt.vlines(0, -2, 2, color='k', linestyles=':')
plt.vlines(1, -2, 2, color='k', linestyles=':')
# 학습 데이터는 원, 테스트데이터는 십자, 클래스 0은 파랑, 클래스 1은 빨강으로 표시
draw(x_train, y_train, 'o')
draw(x_test, y_test, '+')
plt.xlim([-1, 2])
plt.ylim([-1, 2])
plt.show()

## Pytorch에서 사용하기 위해 Tensor로 변환

In [ ]:
x_train = torch.FloatTensor(x_train)
print(x_train.shape)
x_test = torch.FloatTensor(x_test)
y_train = torch.FloatTensor(y_train)
y_test = torch.FloatTensor(y_test)

In [ ]:
class NeuralNet(torch.nn.Module):
        def __init__(self, input_dim, hidden_dim):
            super(NeuralNet, self).__init__()
            self.input_dim = input_dim
            self.hidden_dim  = hidden_dim
            
            '''
            여기에 코드작성
            '''
            
        def forward(self, x):
            '''
            여기에 코드작성
            '''
            return x 

In [ ]:
learning_rate = 3e-2
epochs = 2000
model = NeuralNet(2, 5)
criterion = torch.nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

In [ ]:
model(torch.tensor([0.1,0.1]))

In [ ]:
list(model.parameters())

In [ ]:
[p.grad for p in model.parameters()]

In [ ]:
model.eval()
test_loss_before = criterion(model(x_test).squeeze(), y_test)
print('Before Training, test loss is {}'.format(test_loss_before.item()))

In [ ]:
model.train()
optimizer.zero_grad()
train_output = model(x_train)
train_loss = criterion(train_output.squeeze(), y_train)
train_loss.backward()
optimizer.step()

In [ ]:
import os
path = './ckpt'
if not os.path.isdir(path):
    os.mkdir(path)

In [ ]:
for epoch in range(0, epochs+1):
    model.train()
    optimizer.zero_grad()
    train_output = model(x_train)
    train_loss = criterion(train_output.squeeze(), y_train)
    if epoch % 100 == 0:
        print(f'{epoch:<4} | Train loss: {train_loss.item():.6f}')
        torch.save(model.state_dict(), './ckpt/model_epoch{}.pt'.format(epoch))
    train_loss.backward()
    optimizer.step()

In [ ]:
model.eval()
test_loss = criterion(model(x_test).squeeze(), y_test)
print('After Training, test loss is {}'.format(test_loss.item()))

In [ ]:
torch.save(model.state_dict(), './ckpt/model.pt')
model.state_dict()

In [ ]:
new_model = NeuralNet(2, 5)
new_model.load_state_dict(torch.load('./ckpt/model.pt'))
new_model.eval()
print('벡터 [0, 1]의 레이블은?')
print(new_model(torch.FloatTensor([0,1])).item())

In [ ]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round((y_pred)).squeeze()
    
    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100).item()
    return acc

In [ ]:
new_model.eval()
print('Predict :', torch.round(new_model(x_test[:10])).squeeze().tolist())
print('Answer  :', y_test[:10].tolist())
print(binary_acc(new_model(x_test), y_test))

In [ ]:
for i in range(0, epochs+1, 100):
    new_model = NeuralNet(2, 5)
    new_model.load_state_dict(torch.load('./ckpt/model_epoch{}.pt'.format(i)))
    new_model.eval()
    print(f'Epoch {i:<4} | Test Accuracy: {binary_acc(new_model(x_test), y_test)}%')